In [2]:
import numpy as np
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import csv
from datetime import date, datetime
import datetime
from tslearn.barycenters import dtw_barycenter_averaging
from tslearn.clustering import TimeSeriesKMeans
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import RobustScaler
from scipy.spatial.distance import euclidean
from sklearn.feature_selection import f_regression
from statsmodels.tsa.stattools import adfuller
from sklearn.manifold import TSNE
from DBCV import DBCV
import hdbscan
import csv
import multiprocessing
import itertools
import operator
import math
from dtaidistance import dtw
import seaborn as sns
import elevation
import json
plt.style.use('fivethirtyeight')
from osgeo import gdal 
from subprocess import Popen
import simplekml
import copy
from sklearn.cluster import DBSCAN
import numpy.ma as ma
from numpy import linalg as LA

In [24]:
class Data():
    
    def __init__(self, data_path, filename1, filename2, pc=0.3):
        self.load(data_path, filename1, filename2, pc)
        
    def set_latitudes(self, latitudes):
        self.latitudes = latitudes
        
    def set_longitudes(self, longitudes):
        self.longitudes = longitudes
        
    def set_topo(self, topo):
        self.topo = topo
        
    def set_ns_mean_velocities(self, velocities):
        self.ns_mean_velocities = velocities
        
    def set_ew_mean_velocities(self, velocities):
        self.ew_mean_velocities = velocities
    
    def set_dates(self, dates):
        self.dates = dates
    
    def set_ns_displacements(self, ns_displacements):
        self.ns_displacements = ns_displacements
        
    def set_ew_displacements(self, ew_displacements):
        self.ew_displacements = ew_displacements
        
    def load(self,  data_path, filename1, filename2, pc):
        ns_displacements, ew_displacements, booleans = [], [], []
        ns_infos, ns = self.load_component(data_path, filename1)
        ew_infos, ew = self.load_component(data_path, filename2)
        m = len(ns[0])
        
        for n, components in enumerate(zip(ns, ew)):
            if ns[n].isnull().sum().sum() / m < pc:
                ns_displacements.append(components[0].interpolate(limit_direction='both', inplace=False)['displacement'].values)
                ew_displacements.append(components[1].interpolate(limit_direction='both', inplace=False)['displacement'].values)
                booleans.append(False)
            else:
                booleans.append(True)
            
        self.set_latitudes(ma.array(ns_infos['Lat'].values, mask = booleans).compressed())
        self.set_longitudes(ma.array(ns_infos['Lon'].values, mask = booleans).compressed())
        self.set_topo(ma.array(ns_infos['Topo'].values, mask = booleans).compressed())
        self.set_ns_mean_velocities(ma.array(ns_infos['Vel'].values, mask = booleans).compressed())
        self.set_ew_mean_velocities(ma.array(ew_infos['Vel'].values, mask = booleans).compressed())
        self.set_ns_displacements(np.array(ns_displacements))
        self.set_ew_displacements(np.array(ew_displacements))
        self.set_dates(ns[0].index)
        
    #np.count_nonzero(np.isnan(data))
    def load_image_correlation(self, data_path, ns_fi, ew_filename):
        df_ns, df_ns_ts = self.load_component(data_path, ns_filename)
        df_ew, df_ew_ts = self.load_component(data_path, ew_filename)
        df_ew.rename(columns={'Vel': 'Vel_ew'}, inplace=True)
        df_ns.rename(columns={'Vel': 'Vel_ns'}, inplace=True)
        geo = pd.concat([df_ew[['id', 'Lat','Lon','Topo','Vel_ew']], df_ns[['Vel_ns']]], axis=1)
        return geo, df_ns_ts, df_ew_ts
        
    def load_component(self, data_path, filename):
        
        # numéro de la ligne ou commence les données
        num_start = 44
        # numéro de la ligne ou se trouve la liste des dates
        num_list_dates = 40
        # attributs présent dans les données
        columns = ['id', 'Lat','Lon', 'Topo', 'Vel', 'Coer',' CosN', 'CosE', 'CosU']
        # dictionnaire stockant les données
        data = {column: [] for column in columns}
        # liste des dates 
        indexes = []
        # series temporelles
        series = []
        # liste de dataframes
        df_series = []
        
        with open(data_path + '/' + filename) as csv_file:
            csv_reader = csv.reader(csv_file, delimiter=",")
            line_count = 1 
            for row in csv_reader:
                if line_count == num_list_dates:
                    indexes = [row[0].split(' ')[1]] + row[1:]
                if line_count >= num_start:
                    # extraction des premiers attributs
                    for i in range(len(columns)):
                        data[columns[i]].append(row[i])
                    # extraction de l'attribut TS(série temporelle)
                    series.append([float(v) for v in row[len(columns):]])
                line_count  += 1
            if len(indexes) != len(series[0]):
                print('Erreur : Les indexes et les valeurs ne correspondent pas')
            # convertir les index en date
            indexes = [d.strip()[0:8] for d in indexes]
            # créer une liste de dataframes, chacun contenant une série temporelle
            for serie in series:
                tmp_serie = pd.DataFrame({'displacement': pd.Series(serie, index=pd.DatetimeIndex(indexes))})
                tmp_serie.sort_index(inplace=True)
                df_series.append(tmp_serie)
            # creer un dataframe pour les autres attributs
            df = pd.DataFrame(data)
            for column in df.columns:
                df[column] = pd.to_numeric(df[column], errors='coerce')
            df.set_index('id')
        return df, df_series

In [25]:
class Displacement():
    def __init__(self):
        pass
    
    def compute_velocity(self, displacement):
        return 0
    
    def test(self):
        return 0
    
    def is_mean_velocity_significant(self, displacements, days):
        # extraire X et y
        X = StandardScaler().fit_transform(displacements)
        # calculer la p-value de la regression lineaire
        _, pval = f_regression(X, days)
        return  pval[0]
    
    def compute_inst_vel(self, displacements, dates):
        diff_displacements, durations = self.prepare(displacements, dates)
        return np.divide(diff_displacements, durations)
    
    # TODO
    def is_stationary(self, displacement, alpha, freq='D'):
        resampled = displacement.resample(freq)
        interpolated = upsampled.interpolate(method='linear')
        return self.compute_adfuller(interpolated) < alpha
    
     # TODO
    def smooth(self, displacement, factor):
        std = math.sqrt(serie.var())
        for i in range(len(serie)):
            if abs(serie.iloc[i].displacement) > ampl*std:
                serie.iloc[i, serie.columns.get_loc('displacement')]= np.nan
        return serie.interpolate(limit_direction='both', inplace=False)
    
    def prepare(self, displacement, dates):
        durations = np.diff(dates) / np.timedelta64(1,'D')
        return self.transform_displacement(displacement), self.transform_date(dates)
    
    def transform_date(self, dates):
        temp = np.diff(np.array(dates))
        shifted = np.array([np.timedelta64(1, 'D') for n in range(temp.shape[0])])
        shifted[0:-1] = temp[1:]
        return (shifted + temp)[:-1] / np.timedelta64(1,'D')
    
    def transform_displacement(self, displacement):
        temp = np.diff(np.array(displacement))
        shifted = np.zeros(temp.shape[0])
        shifted[0:-1] = temp[1:]
        return np.add(temp, shifted)[:-1]
    
    def transform_dates_for_lnreg(self, dates):
        lnr_dates = np.cumsum( np.diff(data.dates) /  np.timedelta64(1,'D') )
        return np.insert(lnr_dates, 0, 0, axis=0)

In [28]:
class DataFromImageCorrelation():
    
    def __init__(self, data, raster_folder_name, dem_filename, pc=0.4, alpha=0.05, ref='wgs84'):
        self.data  = data
        self.compute_slope_map(raster_folder_name, dem_filename)
        self.compute_vlm_std()
        self.n_cores = multiprocessing.cpu_count()
        self.pc = pc
        self.ref = ref
        self.alpha = alpha
        self.velocities = []
        self.mask = np.array([])
    
    def compute_vel(self, ns_component, ew_component):
        return math.sqrt(ns_component * ns_component + ew_component * ew_component)
    
    def is_ns_mean_velocity_significant(self, n):
        displacement = self.data.ns_displacements[n].reshape(-1, 1)
        dates = Displacement().transform_dates_for_lnreg(self.data.dates)
        return Displacement().is_mean_velocity_significant(displacement, dates) < self.alpha
    
    def is_ew_mean_velocity_significant(self, n):
        displacement = self.data.ew_displacements[n].reshape(-1, 1)
        dates = Displacement().transform_dates_for_lnreg(self.data.dates)
        return Displacement().is_mean_velocity_significant(displacement, dates) < self.alpha
    
    def is_mean_velocity_significant(self, n):
        return self.is_ns_mean_velocity_significant(n) and self.is_ew_mean_velocity_significant(n)
    
    def is_moving(self, n, factor):
        ns_vel = data.ns_mean_velocities[n]
        ew_vel = data.ew_mean_velocities[n]
        return self.compute_vel(ns_vel, ew_vel) > factor * self.sigma
    
    def filter_by(self, factor, min_slope=5):
        vfunc = np.vectorize(self.is_to_select)
        self.mask = vfunc(np.arange(self.data.latitudes.shape[0]), factor, min_slope)

    def is_to_select(self, n, factor, min_slope):
        return (self.is_moving(n, factor) and
               self.is_ns_mean_velocity_significant(n) and
                self.is_ew_mean_velocity_significant(n))
    
    def filter_by_linear_reg(self):
        pass
                
    def compute_vlm_std(self):
        vfunc = np.vectorize(self.compute_vel)
        ns_velocities = self.data.ns_mean_velocities
        ew_velocities = self.data.ew_mean_velocities
        self.sigma = np.std(vfunc(ns_velocities, ew_velocities))
        
    def compute_inst_vels(self, series):
        with multiprocessing.Pool(self.n_cores) as p:
            results = p.map(Displacement().compute_inst_vel, series)
            return results
        
    def compute_velocities(self):
        if self.mask.shape[0] > 0:
            ns_displacements = self.data.ns_displacements[~np.array(self.mask)]
            ew_displacements = self.data.ew_displacements[~np.array(self.mask)]
            ns_vels = self.compute_inst_vels(ns_displacements)
            ew_vels = self.compute_inst_vels(ew_displacements)
            vfunc = np.vectorize(self.compute_vel)
            self.velocities = np.array([vfunc(ns, ew) for ns, ew in zip(ns_displacements, ew_displacements)])
                    
    def set_slope_map_path(self, slope_map_path):
        self.slope_map_path = slope_map_path
        
    def load_raster(self, raster_folder_name, raster_filename):
        return gdal.Open(raster_folder_name + '/' +  raster_filename)
        
    def compute_slope_map(self, raster_folder_name, dem_name):
        dem = None
        slope_map = None
        slope_map_name = dem_name.split('.')[0] + '_' + 'slope_map.tif'
        slope_map_path = raster_folder_name + '/' + slope_map_name
        if not os.path.isfile(slope_map_path):
            dem = gdal.Open(raster_folder_name + '/'+ dem_name)
            slope_map = gdal.DEMProcessing(slope_map_path, dem, 'slope', computeEdges = True)
        self.set_slope_map_path(slope_map_path)

# Application

## Chargement des données

In [29]:
DATA_PATH  = './donnees' 
filename1  = 'MM_TIO_NS_31TGK_20151227_to_20200906.csv'
filename2  = 'MM_TIO_EW_31TGK_20151227_to_20200906.csv'

In [30]:
data = Data(DATA_PATH, filename1, filename2)

KeyboardInterrupt: 

## Calcul des profils de vitesse

In [ ]:
raster_folder_path = 'rasters'
dem_filename =  '31TGK_copernicus_dem.tif'

In [ ]:
img_correlation = DataFromImageCorrelation(data, raster_folder_path, dem_filename)

In [294]:
img_correlation.sigma

0.0015833768879708388

In [23]:
img_correlation.filter_by(factor=3)

NameError: name 'img_correlation' is not defined

In [935]:
 def plot_disp_ns_ew(ns_ts, ew_ts):

        fig, ax = plt.subplots(2, 1, figsize=(15,10))

        ax[0].plot(ns_ts, color='blue', label='displacement (m)', marker='o', linewidth=2)
        ax[0].set_title('NS cumulative displacement')
        ax[0].set_xlabel('time')
        ax[0].set_ylabel('displacement')
        ax[0].legend()

        ax[1].plot(ew_ts, color='orange', label='displacement (m)', marker='o', linewidth=2)
        ax[1].set_title('EW cumulative displacement')
        ax[1].set_xlabel('time')
        ax[1].set_ylabel('displacement')
        ax[1].legend()
        plt.savefig('displacement.png')
        fig.tight_layout()
        plt.show()
        

## Analyse en composantes princpales(PCA) et décomposition en composantes indépendantes (ICA)

In [355]:
from sklearn.decomposition import PCA, FastICA

In [693]:
n_components = None
pca = PCA(n_components=n_components, random_state=0)
ica = FastICA(n_components=n_components, random_state=0)

In [926]:
def compute_decomposition(data, n_components=20, tol=0.005, random_state=0):
    
    sources_ica = compute_ica_decomposition(data)
    sources_pca = compute_pca_decomposition(data)
    
    return sources_ica, sources_pca

def compute_ica_decomposition(data, n_components=20, tol=0.005, random_state=0):
    
    # normalisation des données
    X = normalize(data).T
    
    # décomposition en composantes indépendantes (ICA)
    ica = FastICA(n_components=n_components, random_state=random_state, tol=tol)
    sources_ica = ica.fit_transform(X)
    
    return sources_ica

def compute_pca_decomposition(data, n_components=20, tol=0.005, random_state=0):
    
    # normalisation des données
    X = normalize(data).T
    
     # décompositions en composantes principales (PCA)
    pca = PCA(n_components=n_components, random_state=random_state)
    sources_pca = pca.fit_transform(X)
    
    return sources_pca

def plot_components(data, output, ncols=3, colormap='tab20', figsize=(25,15)):
    n = data.shape[1]
    nrows = int(n / ncols) if n % ncols == 0 else int(math.ceil(n / ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)
    cmap = plt.get_cmap(colormap)
    colors = [cmap(i) for i in np.linspace(0, 1, n)]
    for i in range(n):
        axes[int(i / ncols), i % ncols].plot(data[:, i], color=colors[i])
        axes[int(i / ncols), i % ncols].set_title('component %d' % (i+1))
    plt.savefig(output + '.png')
    fig.tight_layout()
    plt.show()

In [924]:
ns_sources_ica = compute_ica_decomposition(data.ns_displacements, n_components=20)
ew_sources_ica = compute_ica_decomposition(data.ew_displacements, n_components=20)
ns_sources_pca = compute_pca_decomposition(data.ns_displacements, n_components=20)
ew_sources_pca = compute_pca_decomposition(data.ew_displacements, n_components=20)

In [ ]:
plot_components(ns_sources_ica, 'ns_ica')
plot_components(ew_sources_ica, 'ew_ica')
plot_components(ns_sources_pca, 'ns_pca')
plot_components(ew_sources_pca, 'ew_pca')

# Sélection des sources

# Remarques

* les deux méthodes ne renvoient pas la même amplitude 
* amplitude plus faible avec ICA
* les sources avec PCA sont obtenues dans la base orthogonal formée par les composantes principales


In [21]:

def generate_kml_file(latitudes, longitudes, color, kml=None, filename='visualization', generate=True):
    
    url = 'http://maps.google.com/mapfiles/ms/micons/'
    kml = simplekml.Kml()
    for n, coords in enumerate(zip(latitudes, longitudes)):
        pnt = kml.newpoint(description=str(n), coords=[(coords[0], coords[1])])
        pnt.iconstyle.icon.href = url + color + '-dot.png'
    if generate:
        kml.save(filename + '.kml')
        return  None
    else:
        return kml
        

In [22]:
generate_kml_file(data.latitudes, data.longitudes,'green', 'geo')